In [1]:
!pip install torch torch_geometric

  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
     ---------------------------------------- 0.0/63.1 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.1 kB 330.3 kB/s eta 0:00:01
     ------------------------------ ------- 51.2/63.1 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 63.1/63.1 kB 484.6 kB/s eta 0:00:00
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 0.0/74.1 kB ? eta -:--:--
     -----------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from torch_geometric.data import Data
import pandas as pd

# Load the data

In [6]:
df = pd.read_csv('..\\get_lineup_target_score\\nba_with_lineup_score.csv')

# Step 1: Map each unique lineup to an integer index (node ID)

In [10]:
lineups = pd.unique(df[['home_lineup', 'away_lineup']].values.ravel())
lineup_to_id = {lineup: i for i, lineup in enumerate(lineups)}

# Step 2: Build edge list with direction based on lineup_score

In [11]:
edge_index = []

for _, row in df.iterrows():
    home = row['home_lineup']
    away = row['away_lineup']
    home_score = row['lineup_score']

    # Estimate away score if not directly available
    # For now, let's assume both teams get the same formula
    # You may want to calculate a true 'away_lineup_score' in your preprocessing
    away_score = row.get('away_lineup_score', 0)

    if home_score > away_score:
        edge_index.append([lineup_to_id[away], lineup_to_id[home]])
    elif away_score > home_score:
        edge_index.append([lineup_to_id[home], lineup_to_id[away]])

# Convert to tensor (shape: [2, num_edges])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Step 3: Create node features

In [12]:
# Placeholder: use a simple zero vector for each lineup
# Replace with actual stats (from nba_api) per lineup
num_nodes = len(lineup_to_id)
x = torch.zeros((num_nodes, 10))  # e.g., 10 features per lineup

# Step 4: Create PyG Data object

In [13]:
data = Data(x=x, edge_index=edge_index)

print(data)

Data(x=[43136, 10], edge_index=[2, 46035])
